In [7]:
import os
import sys
import time
import pandas as pd
import geopandas as gpd
import arcpy
import numpy as np

In [2]:
# show all columns
pd.options.display.max_columns = None

In [3]:
# Read in hdf5 
store = pd.HDFStore('.\\Data\\remm_data_2015_base_year_02082019.h5')
tables = list(store.keys())
tables

['/buildings',
 '/buildings_for_estimation',
 '/buildings_for_estimation_grouped',
 '/employment_controls',
 '/household_controls',
 '/household_for_estimation',
 '/households',
 '/households_for_estimation',
 '/households_for_estimation1',
 '/jobs',
 '/parcels',
 '/travel_data',
 '/valid_parcels',
 '/zoning',
 '/zoning_base_line',
 '/zoning_baseline',
 '/zoning_for_parcels']

In [17]:
# Load parcels into spatial dataframe
parcels_shp = gpd.read_file(r'.\data\parcels_county_id.shp')

In [5]:
# get the parcels table from the store
parcels = store.get('parcels').reset_index()
del parcels['Unnamed: 0']
parcels.head(5)

,parcel_id,county_id,zone_id,parcel_acres,land_value,x,y,elevation,fwy_exit,airport,rail_depot,stream,trail,university,shape_area,volume_one_way,volume_two_way,airport_distance,fwy_exit_dist,raildepot_dist,university_dist,trail_dist,stream_dist,train_station,rail_stn_dist,bus_rte_dist,bus_stop,bus_stop_dist,volume_two_way_nofwy,distsml_id,distmed_id,distlrg_id,parent_parcel,CO_NAME,parcel_id_REMM,county_taz_id,utmxi,utmyi,city,zonal_ppa
0,741871,4,2111,0.197968,35700.00,1531239.186,7330722.151,1447.537450,0.0,0.0,0.0,1.0,0.0,0.0,8.623484e+03,992.0,1942.0,18.02655,0.85831,12.61234,14.73993,0.09764,0.0,0.0,8.748286,2382.31,0.0,0.882466,1942.0,487,38,13,741871,UTAH,741871,492111,424331.7633,4477427.732,Lehi,176926.527483
1,579822,4,2873,37.717834,56614.32,1535437.548,7341595.932,1657.284652,0.0,0.0,0.0,1.0,0.0,0.0,1.642989e+06,3564.0,7135.0,19.48828,1.32984,10.63629,15.94851,0.23207,0.0,0.0,6.691816,4930.50,0.0,1.164110,7135.0,0,0,0,579822,UTAH,579822,492873,425629.9856,4480733.739,Utah County,140869.334754
2,579853,4,2117,6.791235,81500.00,1534101.235,7334630.179,1657.284652,0.0,0.0,0.0,1.0,0.0,0.0,2.958262e+05,0.0,0.0,18.42054,1.66976,11.91035,15.00142,0.55081,0.0,0.0,7.994607,6770.95,0.0,1.661999,0.0,487,38,13,579853,UTAH,579853,492117,425210.6619,4478613.554,Lehi,199096.575130
3,640185,4,2073,1.372797,112100.00,1538253.186,7322787.258,1445.761763,0.0,0.0,0.0,1.0,0.0,0.0,5.979902e+04,2495.0,5043.0,16.06753,0.60111,14.23267,12.73670,0.04966,0.0,0.0,10.277766,2627.34,0.0,0.548186,5043.0,482,38,13,640185,UTAH,640185,492073,426455.1775,4474997.689,Lehi,169098.553773
4,682698,4,2119,0.163751,43000.00,1548648.398,7336198.043,1766.927829,0.0,0.0,0.0,1.0,0.0,0.0,7.132996e+03,3632.0,7133.0,17.61362,3.74880,12.20122,13.84603,0.37882,0.0,0.0,8.210217,9997.32,0.0,2.129948,7133.0,488,39,14,682698,UTAH,682698,492119,429646.0053,4479065.962,Draper,254583.603528


In [6]:
parcels2 = parcels[['parcel_id', 'county_id', 'zone_id', 'parcel_id_REMM', 'CO_NAME']].copy()

In [13]:
# Check if parcel_id and parcel_id_REMM fields are the same. SPOILER: They are
parcels2['pid_check'] = np.where(parcels2['parcel_id'] == parcels2['parcel_id_REMM'], True, False)
parcels2['pid_check'].value_counts()
#parcels2[parcels2['pid_check']==False]

True    845219
Name: pid_check, dtype: int64

In [24]:
parcels_no_county = parcels2[parcels2['county_id']==0]
parcels_no_county = parcels_shp.merge(parcels_no_county, left_on='parcel_id_', right_on='parcel_id', how='inner')
parcels_no_county.to_file(r'.\results\parcels_no_county.shp')

In [14]:
parcels2["county_id"].value_counts()

2    368139
4    250210
1    113643
3    109672
0      3555
Name: county_id, dtype: int64

In [18]:
parcels_shp.columns

Index(['Join_Count', 'TARGET_FID', 'OBJECTID', 'parcel_id', 'parcel_id_',
       'parcel_sqf', 'x', 'y', 'parcel_acr', 'zone_id', 'county_id',
       'county_taz', 'county_nam', 'city', 'NAME', 'FIPS', 'STATEPLANE',
       'GlobalID', 'FIPS_STR', 'geometry'],
      dtype='object')